# Task 2

Reproduce figure 5A (but use blue-white-red color palette instead of green-black-red); make another heatmap of gene expression 

Figure caption:

ILC Molecular Subtypes(A) Three molecular subtype of lobular breast cancer were identified based on differential gene expression and show unique patterns highly expressed genes(n = 1277, SAM FDR = 0, upper panel), minor difference in tumor purity measured by ABSOLUTE, and differences in gene expression signatures measuringproliferation, CD68, Macrophage-associated CSF1, Macrophage–associated TH1, and T Cell Receptor Signaling (lower panel). Proliferation is highest in theproliferative(Pro) andimmune-related(IR) subgroups; macrophage associated signaling is highest inimmune-relatedtumors.

Section describing the figure:

### ILC mRNA Subtypes

Using mRNA-seq expression data from LumA ILC samples
(n = 106), we identified three ILC subtypes termed reactive-like,
immune-related, and proliferative (Figures S5A–S5I, Table S8).
We then used a 3-class ILC subtype classifier (60 genes, Table
S13) to score all ILC samples in the TCGA (n = 127) (Figure 5A)
and METABRIC (Curtis et al., 2012) datasets (Table S12). Our
analyses identified many significant genomic features that distinguished each ILC subtype at the mRNA and protein/phosphoprotein level; but no distinguishing somatic mutations or DNA
copy-number alterations.
Significant analysis of microarray (SAM) analysis (Tusher et al.,
2001) identified 1,277 genes differentially expressed between
ILC subtypes (q = 0) (Figure 5A, Table S8). Of these, 1,005
were highly expressed in reactive-like tumors, which had lower
tumor purity as determined by ABSOLUTE (Carter et al., 2012)
(Figures 5A and S5P), and included genes consistent with
epithelial and stromal-associated signaling including keratin,
kallikrein, and claudin genes as well as the oncogenes EGFR,
MET, PDGFRA, and KIT (Table S8). The remaining 272 genes
were highly expressed in immune-related tumors and include
modulators of immunogenic signaling such as interleukins (IL),
chemokine receptors and ligands, major histocompatibility complex, and tumor necrosis factors, as well as IDO1 and IFNG
(Figure 5A and Table S8). Interestingly, immune activity in this
subset of tumors appears to be predominantly associated with
macrophage-associated signaling as increased levels of CD68
(p < 0.05), macrophage-associated colony stimulating factor
(MacCSF), macrophage-associated TH1 (MacTH1), and T cell
receptor (TCR) gene expression signatures (Iglesia et al., 2014)
were observed in both the TCGA (Figure 5A) and METABRIC
(Figure S5J–S5K) datasets. Finally, proliferative tumors were
defined by low expression of each of these 1,277 genes
(Figure 5A). Intriguingly, in each dataset (Figures 5 and S5K)
proliferative tumors had higher levels of proliferation relative
to reactive-like tumors (TCGA: p = 3.3E-09; METABRIC: p =
0.018) and slightly higher or equivalent levels compared to
immune-related ones (TGCA: p = 0.29; METABRIC: p = 0.008).
Regardless of ILC subtype, ILC tumor proliferation was generally
lower than all IDC subtypes (Figures S5L–S5M).

<img src="img/5A.png" alt="5A" width="300"/>

In [1]:
import pandas as pd
import numpy as np

## load the data

Source: supplementary material to https://www.cbioportal.org/study/summary?id=brca_tcga_pub2015

In [2]:
ilc_classification_df = pd.read_excel("data/1-s2.0-S0092867415011952-mmc9.xlsx").iloc[2:, 2:5].dropna()
ilc_classification_df.columns = ilc_classification_df.iloc[0]
ilc_classification_df = ilc_classification_df.drop(ilc_classification_df.index[0])
ilc_classification_df = ilc_classification_df[(ilc_classification_df["Dataset"] == "TCGA")][["Sample ID", "60 Gene-classifier Class Assignment"]]
ilc_classification_df["Sample ID"] = ilc_classification_df["Sample ID"].apply(lambda x: x.replace(".", "-") + "-01")
ilc_classification_df.head()

2,Sample ID,60 Gene-classifier Class Assignment
3,TCGA-AR-A2LH-01,Reactive-like
4,TCGA-AR-A1AK-01,Reactive-like
5,TCGA-BH-A0HP-01,Reactive-like
6,TCGA-A2-A0EX-01,Reactive-like
7,TCGA-A2-A0YK-01,Reactive-like


In [3]:
ilc_genes_df = pd.read_excel("data/1-s2.0-S0092867415011952-mmc9.xlsx").iloc[2:, 6:8].dropna()
ilc_genes_df.columns = ilc_genes_df.iloc[0]
ilc_genes_df = ilc_genes_df.drop(ilc_genes_df.index[0])
ilc_genes_df = ilc_genes_df.set_index(ilc_genes_df["GID| UID"].str.split("|", expand=True)[1].astype(int))
# ilc_genes_df = ilc_genes_df.drop(columns=["GID| UID"])
ilc_genes_df.columns.name = ""
ilc_genes_df.head()

,GID| UID,Class High Expression (FDR=0)
1,,
342667,STAC2|342667,Reactive-like
3861,KRT14|3861,Reactive-like
5650,KLK7|5650,Reactive-like
6663,SOX10|6663,Reactive-like
118430,MUCL1|118430,Reactive-like


From here on the index of dataframes is the UID of the gene. The shapes of the df expected: 1277 genes and 127 samples, where we got 1277 genes and the expected number of samples. This is due to the fact that gene with Hugo Symbol ANXA8L2 and id 244 is present among the ilc genes, but not in the data_rna_seq_v2_expression_median.txt file.

In [4]:
rna_seq_df = pd.read_csv("data/data_RNA_Seq_v2_expression_median.txt", sep="\t")
gene_id_in_rna_seq = rna_seq_df["Entrez_Gene_Id"]
Hugo_Symbol_in_rna_seq = rna_seq_df["Hugo_Symbol"]
rna_seq_df = rna_seq_df.set_index(rna_seq_df["Entrez_Gene_Id"])
rna_seq_df = rna_seq_df.drop(columns=["Entrez_Gene_Id", "Hugo_Symbol"])
rna_seq_df = rna_seq_df.merge(ilc_genes_df, right_index=True, left_index=True)
ilc_expression = rna_seq_df[ilc_classification_df["Sample ID"]]
ilc_expression.shape

(1276, 127)

In [5]:
ilc_genes_df[ilc_genes_df.merge(rna_seq_df, right_index=True, left_index=True, how="left").iloc[:, 5].isna()]

,GID| UID,Class High Expression (FDR=0)
1,,
244,ANXA8L2|244,Reactive-like


In [6]:
(gene_id_in_rna_seq == 244).sum()

0

In [7]:
(Hugo_Symbol_in_rna_seq == "ANXA8L2").sum()

0

In [8]:
# scale to allow log scaling
ilc_expression[ilc_expression == 0] = 0.000001
# log scaling
ilc_expression = np.log(ilc_expression)

/Users/szalata/anaconda3/envs/ada/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/szalata/anaconda3/envs/ada/lib/python3.7/site-packages/pandas/core/frame.py:3530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [9]:
ilc_classification_df["60 Gene-classifier Class Assignment"] = pd.Categorical(ilc_classification_df["60 Gene-classifier Class Assignment"])
ilc_classification_df["subtype_code"] = ilc_classification_df["60 Gene-classifier Class Assignment"].cat.codes

Export to R to perform SAM

In [10]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

In [11]:
with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_classification = ro.conversion.py2rpy(ilc_classification_df[["subtype_code"]])
    r_from_pd_expression = ro.conversion.py2rpy(ilc_expression)
r_from_pd_classification.to_csvfile("data/ilc_classification_df.csv")
r_from_pd_expression.to_csvfile("data/ilc_expression_df.csv")

<rpy2.rinterface_lib.sexp.NULLType object at 0x103fb33c0> [RTYPES.NILSXP]

### Here execute heatmap5A.R and save the heatmap

subtypes color map:
- black: Reactive-like
- red: Immune-related
- green: Proliferative

<img src="img/heatmap5A.png" alt="5A" width="500"/>